In [17]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.metrics import classification_report

# Data Augmentation Parameters
batchsize = 64  # Increase batch size for faster training
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2,
                                   height_shift_range=0.2, validation_split=0.2)

# Data Generators
train_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                               target_size=(224, 224), batch_size=batchsize,
                                               class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\train',
                                                    target_size=(224, 224), batch_size=batchsize,
                                                    class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\data\test',
                                             target_size=(224, 224), batch_size=batchsize,
                                             class_mode='categorical')

# Base model
bmodel = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Model head
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(128, activation='relu')(hmodel)  # Increase number of neurons
hmodel = Dropout(0.5)(hmodel)  # Increase dropout rate to reduce overfitting
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

# Combined model
model = Model(inputs=bmodel.input, outputs=hmodel)

# Freeze layers in base model
for layer in bmodel.layers:
    layer.trainable = False

# Callbacks
checkpoint_path = r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels_resnet.keras'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)

callbacks = [checkpoint, earlystop, learning_rate]

# Adjust Learning Rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Reduce learning rate for smoother convergence
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning (unfreeze some layers for fine-tuning)
for layer in bmodel.layers[-30:]:
    layer.trainable = True

# Train the model with increased number of epochs
model.fit(train_data,
          steps_per_epoch=train_data.samples // batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples // batchsize,
          callbacks=callbacks,
          epochs=30)

# Load the saved model
#model = tf.keras.models.load_model(r'C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels.keras')

# Evaluate the model on the training data generator
loss_tr, acc_tr = model.evaluate(train_data)

# Print the training accuracy and loss
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)

# Evaluate the model on the validation data generator
loss_vr, acc_vr = model.evaluate(validation_data)

# Print the validation accuracy and loss
print("Validation Accuracy:", acc_vr)
print("Validation Loss:", loss_vr)

# Evaluate the model on the test data generator
loss_test, acc_test = model.evaluate(test_data)

# Print the test accuracy and loss
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)

# Get predictions for the test data
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Map class indices to class labels
class_labels = list(test_data.class_indices.keys())

# Get true labels for the test data
true_classes = test_data.classes

# Generate classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the classification report
print(report)


Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Epoch 1/30


C:\Users\rajat\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5001 - loss: 0.7857
Epoch 1: val_loss improved from inf to 0.69311, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels_resnet.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 4478s 5s/step - accuracy: 0.5001 - loss: 0.7856 - val_accuracy: 0.5060 - val_loss: 0.6931 - learning_rate: 1.0000e-04
Epoch 2/30


C:\Users\rajat\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 0.69311
849/849 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4667 - val_loss: 0.6934 - learning_rate: 1.0000e-04
Epoch 3/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5091 - loss: 0.6931
Epoch 3: val_loss improved from 0.69311 to 0.69309, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels_resnet.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 4300s 5s/step - accuracy: 0.5091 - loss: 0.6931 - val_accuracy: 0.5059 - val_loss: 0.6931 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 0.69309 to 0.69265, saving model to C:\Users\rajat\OneDrive\Desktop\drowsiness\Project\trainmodels_resnet.keras
849/849 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5333 - val_loss: 0.6927 - learning_rate: 1.0000e-04
Epoch 5/30
849/849 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5065 - loss: 0.6931
Epoch 5: val_loss did not imp

C:\Users\rajat\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rajat\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rajat\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
